In [3]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # or comment out if you want interactive
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa
from matplotlib.animation import FuncAnimation, FFMpegWriter

# ------------------------------------------------------------
# 1. Load seismicity catalog
# ------------------------------------------------------------

data_frame = pd.read_excel(
    "RosemanowesDataPhase2A.xls",
    sheet_name="P only Source Parameters",
    header=4
)

# Time (hours); adjust column indices to your table if necessary
time_hours = np.array(data_frame.iloc[:, 1])  # event time in hours

# Epicentre coordinates [X, Y, Z]; adjust columns if needed
# X: East, Y: North, Z: Depth (positive downward is fine)
X = np.array(data_frame.iloc[:, 4])  # East
Y = np.array(data_frame.iloc[:, 5])  # North
Z = -np.array(data_frame.iloc[:, 6])  # Depth (m), minus sign to convert to positive downward

epi = np.column_stack([X, Y, Z])

# Moment (or magnitude proxy) for symbol size
pmoment = np.array(data_frame.iloc[:, 12])

# crude scaling from moment to plot size (tune as needed)
mag_proxy = np.log10(np.maximum(pmoment, 1e10))  # avoid log(0)
size_events = 5 + 20 * (mag_proxy - mag_proxy.min()) / (
    mag_proxy.max() - mag_proxy.min() + 1e-9
)

# ------------------------------------------------------------
# 2. Time relative to injection start, convert to seconds
# ------------------------------------------------------------

t0_hours = 3513.0  # injection start
t_rel_hours = time_hours - t0_hours

# Keep only events after t0
mask = t_rel_hours >= 0.0
t_rel_hours = t_rel_hours[mask]
epi = epi[mask, :]
size_events = size_events[mask]

t_rel_sec = t_rel_hours * 3600.0

# ------------------------------------------------------------
# 3. Principal coordinate system (PCA) - orientation of the cloud
# ------------------------------------------------------------

coords_centered = epi - epi.mean(axis=0, keepdims=True)
C = np.cov(coords_centered.T)
eigvals, eigvecs = np.linalg.eigh(C)

# sort eigenvalues descending
idx = np.argsort(eigvals)[::-1]
eigvecs = eigvecs[:, idx]

# Rotation matrix: columns are directions of principal axes (x1,x2,x3) in (X,Y,Z)
R = eigvecs
mean_xyz = epi.mean(axis=0)

# coords in principal system: [x1,x2,x3] = R^T * (X - mean)
coords_principal = coords_centered.dot(R)
x1_all = coords_principal[:, 0]
x2_all = coords_principal[:, 1]
x3_all = coords_principal[:, 2]

# ------------------------------------------------------------
# 4. Animation setup
# ------------------------------------------------------------

t_min = t_rel_sec.min()
t_max = t_rel_sec.max()
n_frames = 120  # number of frames in the movie
frame_times = np.linspace(t_min, t_max, n_frames)

# Axis limits (in original coordinates)
margin = 50.0
x_all = epi[:, 0]
y_all = epi[:, 1]
z_all = epi[:, 2]  # depth

xlim = (x_all.min() - margin, x_all.max() + margin)
ylim = (y_all.min() - margin, y_all.max() + margin)
zlim = (z_all.min() - margin, z_all.max() + margin)

# Create figure
fig = plt.figure(figsize=(8, 7))  # slightly smaller to reduce blank margins
ax = fig.add_subplot(111, projection='3d')

# Reduce white space around axes
plt.subplots_adjust(left=0.00, right=1, bottom=0, top=1)

# Axis labels:
ax.set_xlabel('X (m): East', fontsize=12)
ax.set_ylabel('Y (m): North', fontsize=12)
ax.set_zlabel('Z (m): Depth', fontsize=12)

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_zlim(zlim)

ax.view_init(elev=20, azim=135)  # viewing angle

# Plot semi-transparent surface at Z = 0
plane_res = 30
X_plane = np.linspace(xlim[0], xlim[1], plane_res)
Y_plane = np.linspace(ylim[0], ylim[1], plane_res)
X_plane, Y_plane = np.meshgrid(X_plane, Y_plane)
Z_plane = np.zeros_like(X_plane)  # Z=0 plane

ax.plot_surface(
    X_plane, Y_plane, Z_plane,
    alpha=0.35, color='gray', edgecolor='none'
)

# Initial scatter for events (empty)
scatter_events = ax.scatter([], [], [], s=[], c='k', alpha=0.8)

# For the ellipsoid representing the reservoir
ellipsoid_wire = [None]

# Ellipsoid parameter grid (for drawing the surface)
u_ell = np.linspace(0, 2 * np.pi, 20)
v_ell = np.linspace(0, np.pi, 20)
U_ell, V_ell = np.meshgrid(u_ell, v_ell)

# Create a text object for the "title" inside the axes, lower than usual
# Use axis coordinates (0..1) to place it
title_text = ax.text2D(
    0.5, 0.94,
    "",  # will be updated in animation
    transform=ax.transAxes,
    ha='center', va='top', fontsize=10
)


def best_fit_ellipsoid_points(mask_t, quantile=0.999):
    """
    Compute a best-fit ellipsoid that encloses most events in the current frame.

    - Use principal coordinates (PCA already computed globally).
    - For the subset of events with mask_t, compute semi-axes as the
      given quantile of |x1|, |x2|, |x3|.
    - Return ellipsoid surface in original coordinates.
    """
    if not np.any(mask_t):
        return None, None, None

    # Principal coordinates for active events
    x1 = x1_all[mask_t]
    x2 = x2_all[mask_t]
    x3 = x3_all[mask_t]

    # Semi-axes from quantile of absolute values
    a1 = np.quantile(np.abs(x1), quantile)
    a2 = np.quantile(np.abs(x2), quantile)
    a3 = np.quantile(np.abs(x3), quantile)

    # Avoid degenerate ellipsoid early on
    eps = 1e-3
    a1 = max(a1, eps)
    a2 = max(a2, eps)
    a3 = max(a3, eps)

    # Ellipsoid in principal coords: (x1/a1)^2 + (x2/a2)^2 + (x3/a3)^2 = 1
    x1_e = a1 * np.cos(U_ell) * np.sin(V_ell)
    x2_e = a2 * np.sin(U_ell) * np.sin(V_ell)
    x3_e = a3 * np.cos(V_ell)

    pts_principal = np.stack([x1_e, x2_e, x3_e], axis=-1)  # (nu,nv,3)

    # Transform back to original coords: X = mean + R * x_principal
    pts_orig = np.einsum('ij,...j->...i', R, pts_principal) + mean_xyz

    X_e = pts_orig[..., 0]
    Y_e = pts_orig[..., 1]
    Z_e = pts_orig[..., 2]
    return X_e, Y_e, Z_e


def init():
    scatter_events._offsets3d = ([], [], [])
    scatter_events.set_sizes([])
    title_text.set_text("")
    return scatter_events, title_text


def update(frame_idx):
    t_frame = frame_times[frame_idx]

    # Events up to this time
    mask_t = t_rel_sec <= t_frame
    xe = epi[mask_t, 0]
    ye = epi[mask_t, 1]
    ze = epi[mask_t, 2]
    se = size_events[mask_t]

    scatter_events._offsets3d = (xe, ye, ze)
    scatter_events.set_sizes(se)

    # Best-fit ellipsoid for the current cloud
    X_e, Y_e, Z_e = best_fit_ellipsoid_points(mask_t, quantile=0.999)

    # Remove previous ellipsoid wireframe
    if ellipsoid_wire[0] is not None:
        ellipsoid_wire[0].remove()

    if X_e is not None:
        ellipsoid_wire[0] = ax.plot_wireframe(
            X_e, Y_e, Z_e,
            rstride=2, cstride=2,
            color='red', alpha=0.6
        )
    else:
        ellipsoid_wire[0] = None

    # Update "title" lower down inside the axes
    title_text.set_text(
        "Reservoir with seismicity cloud\n"
        f"Time since injection: {t_frame/3600.0:.2f} h"
    )

    return scatter_events, ellipsoid_wire[0], title_text


anim = FuncAnimation(
    fig,
    update,
    init_func=init,
    frames=n_frames,
    interval=20,  # ms between frames
    blit=False
)

# ------------------------------------------------------------
# 6. Save animation
# ------------------------------------------------------------

# Example: GIF via ImageMagick
anim.save("seismicity_reservoir_ellipsoid.gif", writer='imagemagick', fps=5)

# If you want MP4 via ffmpeg instead, uncomment:
# writer = FFMpegWriter(fps=10, bitrate=1800)
# anim.save("seismicity_reservoir_ellipsoid.mp4", writer=writer)

plt.close(fig)


MovieWriter imagemagick unavailable; using Pillow instead.
